In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import xml.etree.ElementTree as ET
import json
from PIL import Image, ImageTk

class XMLViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("StayFitGym - Management Clienți")
        self.root.iconbitmap("dumbell.ico")
        self.root.geometry("700x1000")
        self.tree = None
        self.xml_data = None
        self.xml_file = None
        self.json_data = None
        self.json_file = None
        self.default_font = ("Segoe UI", 10)

        # Incarcare si redimensionare Logo - StayFitGym
        original_logo = Image.open("Logo.png")
        resized_logo = original_logo.resize((200, 40))  # poți ajusta dimensiunea aici
        self.logo_img = ImageTk.PhotoImage(resized_logo)
        logo_label = tk.Label(self.root, image=self.logo_img, bg="#f0f0f0")
        logo_label.pack(anchor="ne", padx=10, pady=5)
        
        self.create_widgets()

    def create_widgets(self):
        # Functionalitate de incarcare XML/JSON
        button_frame = tk.Frame(self.root)
        button_frame.pack(fill=tk.X, pady=5)

        tk.Button(button_frame, text="Încarcă XML", command=self.load_xml).pack(side=tk.LEFT, padx=10)
        tk.Button(button_frame, text="Încarcă JSON", command=self.load_json).pack(side=tk.LEFT, padx=10)

        # Functionalitate de cautare
        search_frame = tk.Frame(self.root)
        search_frame.pack(fill=tk.X, pady=5)
        tk.Label(search_frame, text="Căutare:").pack(side=tk.LEFT)
        self.search_entry = tk.Entry(search_frame)
        self.search_entry.pack(side=tk.LEFT, padx=5)
        tk.Button(search_frame, text="Caută", command=self.search_tree).pack(side=tk.LEFT)

        # Afisarea arborelui
        self.tree_frame = ttk.Frame(self.root)
        self.tree_frame.pack(fill=tk.BOTH, expand=True)
        self.tree = ttk.Treeview(self.tree_frame)
        self.tree.pack(fill=tk.BOTH, expand=True)

        # Functionalitate de adaugare clienti
        self.add_frame = tk.LabelFrame(self.root, text="Adaugă client nou", padx=10, pady=10)
        self.add_frame.pack(fill=tk.X, padx=10, pady=10)

        self.client_fields = {
            "id": tk.Entry(self.add_frame),
            "Nume": tk.Entry(self.add_frame),
            "CNP": tk.Entry(self.add_frame),
            "Telefon": tk.Entry(self.add_frame),
            "Email": tk.Entry(self.add_frame),
            "Abonament_tip": tk.Entry(self.add_frame),
            "Abonament_dataInceput": tk.Entry(self.add_frame),
            "Abonament_dataExpirare": tk.Entry(self.add_frame)
        }

        row = 0
        for label, entry in self.client_fields.items():
            tk.Label(self.add_frame, text=label).grid(row=row, column=0, sticky="e", pady=2)
            entry.grid(row=row, column=1, pady=2, sticky="we")
            row += 1

        tk.Button(self.add_frame, text="Adaugă client", command=self.add_client).grid(row=row, columnspan=2, pady=5)

        # Functionalitate de stergere clienti
        self.add_frame = tk.LabelFrame(self.root, text="Stergere client", padx=3, pady=10)
        self.add_frame.pack(fill=tk.X, padx=10, pady=10)
        tk.Label(self.add_frame, text="ID client de șters").grid(row=row+1, column=0, sticky="e", pady=10)
        self.delete_id_entry = tk.Entry(self.add_frame)
        self.delete_id_entry.grid(row=row+1, column=1, pady=2, sticky="we")
        
        tk.Button(self.add_frame, text="Șterge client", command=self.delete_client).grid(row=row+2, columnspan=2, pady=5)


    def load_xml(self):
        file_path = filedialog.askopenfilename(filetypes=[("XML files", "*.xml")])
        if file_path:
            self.xml_file = file_path
            self.tree.delete(*self.tree.get_children())
            self.xml_data = ET.parse(file_path)
            self.populate_tree(self.xml_data.getroot(), "")

    def load_json(self):
        file_path = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])
        if file_path:
            with open(file_path, 'r', encoding='utf-8') as file:
                try:
                    self.json_data = json.load(file)
                    self.json_file = file_path
                    self.tree.delete(*self.tree.get_children())
                    self.populate_json_tree(self.json_data)
                except Exception as e:
                    messagebox.showerror("Eroare", f"A apărut o eroare la încărcarea fișierului JSON: {e}")


    def populate_tree(self, element, parent):
        text = f"{element.tag}: {element.text.strip() if element.text and element.text.strip() else ''}"
        node = self.tree.insert(parent, "end", text=text)
        for child in element:
            self.populate_tree(child, node)

    def populate_json_tree(self, data, parent=""):
        if isinstance(data, dict):
            for key, value in data.items():
                node = self.tree.insert(parent, "end", text=str(key))
                self.populate_json_tree(value, node)
        elif isinstance(data, list):
            for index, item in enumerate(data):
                node = self.tree.insert(parent, "end", text=f"[{index}]")
                self.populate_json_tree(item, node)
        else:
            self.tree.insert(parent, "end", text=str(data))

    def search_tree(self):
        query = self.search_entry.get().lower()
        for item in self.tree.get_children():
            self.recursive_collapse(item)
        self.search_and_expand(query)

    def recursive_collapse(self, item):
        self.tree.item(item, open=False)
        for child in self.tree.get_children(item):
            self.recursive_collapse(child)

    def search_and_expand(self, query):
        def matches_query(item):
            return query in self.tree.item(item, "text").lower()

        def expand_parents(item):
            parent = self.tree.parent(item)
            while parent:
                self.tree.item(parent, open=True)
                parent = self.tree.parent(parent)

        for item in self.tree.get_children():
            self.recursive_search_expand(item, query, matches_query, expand_parents)

    def recursive_search_expand(self, item, query, match_fn, expand_fn):
        if match_fn(item):
            expand_fn(item)
        for child in self.tree.get_children(item):
            self.recursive_search_expand(child, query, match_fn, expand_fn)

    def add_client(self):
        client_data = {key: entry.get() for key, entry in self.client_fields.items()}
        missing = [k for k, v in client_data.items() if not v]
        if missing:
            messagebox.showerror("Eroare", f"Completează toate câmpurile: {', '.join(missing)}")
            return
    
        # Dacă lucram cu XML
        if self.xml_data is not None:
            root = self.xml_data.getroot()
            clienti = root.find("Clienti")
            if clienti is None:
                clienti = ET.SubElement(root, "Clienti")
    
            client = ET.SubElement(clienti, "Client")
            ET.SubElement(client, "id").text = client_data["id"]
            ET.SubElement(client, "Nume").text = client_data["Nume"]
            ET.SubElement(client, "CNP").text = client_data["CNP"]
            ET.SubElement(client, "Telefon").text = client_data["Telefon"]
            ET.SubElement(client, "Email").text = client_data["Email"]
    
            abonament = ET.SubElement(client, "Abonament")
            ET.SubElement(abonament, "tip").text = client_data["Abonament_tip"]
            ET.SubElement(abonament, "dataInceput").text = client_data["Abonament_dataInceput"]
            ET.SubElement(abonament, "dataExpirare").text = client_data["Abonament_dataExpirare"]
    
            self.write_xml_with_stylesheet()
            messagebox.showinfo("Succes", "Client adăugat cu succes în XML!")
            self.tree.delete(*self.tree.get_children())
            self.populate_tree(self.xml_data.getroot(), "")
    
        # Dacă lucram cu JSON
        elif self.json_data is not None:
            new_client = {
                "id": client_data["id"],
                "Nume": client_data["Nume"],
                "CNP": client_data["CNP"],
                "Telefon": client_data["Telefon"],
                "Email": client_data["Email"],
                "Abonament": {
                    "tip": client_data["Abonament_tip"],
                    "dataInceput": client_data["Abonament_dataInceput"],
                    "dataExpirare": client_data["Abonament_dataExpirare"]
                }
            }
    
            try:
                self.json_data["SalaFitness"]["Clienti"].append(new_client)
                with open(self.json_file, 'w', encoding='utf-8') as f:
                    json.dump(self.json_data, f, indent=2, ensure_ascii=False)
                messagebox.showinfo("Succes", "Client adăugat cu succes în JSON!")
                self.tree.delete(*self.tree.get_children())
                self.populate_json_tree(self.json_data)
            except Exception as e:
                messagebox.showerror("Eroare", f"Eroare la scrierea în JSON: {e}")
        else:
            messagebox.showwarning("Atenție", "Încarcă mai întâi un fișier XML sau JSON.")


    def delete_client(self):
        client_id = self.delete_id_entry.get().strip()
        if not client_id:
            messagebox.showerror("Eroare", "Introduceți un ID.")
            return
    
        # Dacă lucram cu XML
        if self.xml_data is not None:
            root = self.xml_data.getroot()
            clienti = root.find("Clienti")
            if clienti is None:
                messagebox.showinfo("Informație", "Nu există clienți.")
                return
    
            found = False
            for client in clienti.findall("Client"):
                id_elem = client.find("id")
                if id_elem is not None and id_elem.text == client_id:
                    clienti.remove(client)
                    found = True
                    break
    
            if found:
                self.write_xml_with_stylesheet()
                messagebox.showinfo("Succes", f"Clientul cu ID {client_id} a fost șters din XML.")
                self.tree.delete(*self.tree.get_children())
                self.populate_tree(self.xml_data.getroot(), "")
            else:
                messagebox.showwarning("Neidentificat", f"Nu am găsit niciun client cu ID-ul {client_id}.")
    
        # Dacă lucram cu JSON
        elif self.json_data is not None:
            clienti = self.json_data.get("SalaFitness", {}).get("Clienti", [])
            found = False
    
            for i, client in enumerate(clienti):
                if client.get("id") == client_id:
                    del clienti[i]
                    found = True
                    break
    
            if found:
                try:
                    with open(self.json_file, 'w', encoding='utf-8') as f:
                        json.dump(self.json_data, f, indent=2, ensure_ascii=False)
                    messagebox.showinfo("Succes", f"Clientul cu ID {client_id} a fost șters din JSON.")
                    self.tree.delete(*self.tree.get_children())
                    self.populate_json_tree(self.json_data)
                except Exception as e:
                    messagebox.showerror("Eroare", f"Eroare la salvarea fișierului JSON: {e}")
            else:
                messagebox.showwarning("Neidentificat", f"Nu am găsit niciun client cu ID-ul {client_id}.")
    
        else:
            messagebox.showwarning("Atenție", "Încarcă mai întâi un fișier XML sau JSON.")

    def write_xml_with_stylesheet(self):
        xml_str = ET.tostring(self.xml_data.getroot(), encoding="unicode")
    
        # Adaugam declarația XML și linia de stylesheet
        full_xml = (
            '<?xml version="1.0" encoding="utf-8"?>\n'
            '<?xml-stylesheet type="text/xsl" href="afisare_clienti.xsl"?>\n'
            + xml_str
        )
    
        with open(self.xml_file, "w", encoding="utf-8") as f:
            f.write(full_xml)

if __name__ == "__main__":
    root = tk.Tk()
    app = XMLViewerApp(root)
    root.mainloop()
